In [1]:
# modules needed for inline matplotlib and seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()  # set plotting style to seaborn

In [2]:
# modules
import pandas as pd

# sklearn modules
from sklearn.model_selection import StratifiedShuffleSplit

## Load Data File

In [11]:
orig_df = pd.read_csv('../data/munged_data/combined/combined_munged_01012020.csv')
orig_df.head()

,site,bike_type,product_id,href,description,brand,price,msrp,frame_material,model_year,...,handlebar_material,fd_groupset,rd_groupset,cassette_groupset,crankset_material,crankset_groupset,seatpost_material,chain_groupset,shifter_groupset,price_x
0,jenson,mountain,BI178B01,/Evil-Wreckoning-XT-Jenson-USA-Exclusive-Build,Evil Wreckoning XT Jenson USA Exclusive Build,Evil,3499.99,5800.00,NaN,NaN,...,NaN,NaN,shimano deore xt,shimano deore xt,NaN,race face,NaN,shimano 105,shimano deore xt,NaN
1,jenson,mountain,BI001298,/Evil-Following-V1-GX-Eagle-Jenson-USA-Exclusi...,Evil Following V1 GX Eagle Jenson USA Exclusiv...,Evil,3499.99,4844.99,carbon,NaN,...,NaN,NaN,sram gx,sram gx eagle,NaN,sram gx,NaN,sram gx,sram gx,NaN
2,jenson,mountain,BI001373,/Ibis-Ripmo-AF-NX-Eagle-Bike-2020,Ibis Ripmo AF NX Eagle Bike 2020,Ibis,2999.00,2999.00,aluminum,2020.0,...,aluminum,NaN,sram nx,sram nx,steel,sram nx,NaN,sram nx,sram nx,NaN
3,jenson,mountain,BI174B07,/Devinci-Spartan-29-NX12S,Devinci Spartan 29 NX 2019,Devinci,2299.99,3299.00,aluminum,2019.0,...,NaN,NaN,sram nx,sram gx eagle,NaN,sram gx,NaN,sram nx,sram nx,NaN
4,jenson,mountain,BI001166,/Niner-Rip-9-Gx-Eagle-Jenson-USA-Exclusive-Build,Niner Rip 9 Gx Eagle Jenson USA Exclusive Build,Niner,2599.99,4300.00,aluminum,NaN,...,NaN,NaN,sram gx,sram nx,NaN,sram gx,NaN,sram nx,sram gx,NaN


### Quick Glance
Let's take a quick glance to see what we're working with to get a better sense of what we're working with.

#### Info
* There are 9596 samples
* 22 columns of which **price**, **msrp**, **price_x** and **model_year** are numeric data, everthing else is categorical.  **price_x** is a duplicate of price, so need merge into price column and drop.
* Many of the fields have missing values; need to determine how to best handle this.

In [12]:
orig_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9596 entries, 0 to 9595
Data columns (total 22 columns):
site                  9596 non-null object
bike_type             9596 non-null object
product_id            9596 non-null object
href                  9596 non-null object
description           9596 non-null object
brand                 9596 non-null object
price                 9194 non-null float64
msrp                  9596 non-null float64
frame_material        8566 non-null object
model_year            3831 non-null float64
brake_type            8363 non-null object
fork_material         4617 non-null object
handlebar_material    4734 non-null object
fd_groupset           3356 non-null object
rd_groupset           7483 non-null object
cassette_groupset     7183 non-null object
crankset_material     2493 non-null object
crankset_groupset     5008 non-null object
seatpost_material     4264 non-null object
chain_groupset        4712 non-null object
shifter_groupset      6913 non

##### Fill missing price data with price_x values and then drop price_x column.

In [15]:
munged_df = orig_df.copy()
munged_df.price.fillna(munged_df.price_x, inplace=True)
munged_df = munged_df.drop('price_x', axis=1)
munged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9596 entries, 0 to 9595
Data columns (total 21 columns):
site                  9596 non-null object
bike_type             9596 non-null object
product_id            9596 non-null object
href                  9596 non-null object
description           9596 non-null object
brand                 9596 non-null object
price                 9558 non-null float64
msrp                  9596 non-null float64
frame_material        8566 non-null object
model_year            3831 non-null float64
brake_type            8363 non-null object
fork_material         4617 non-null object
handlebar_material    4734 non-null object
fd_groupset           3356 non-null object
rd_groupset           7483 non-null object
cassette_groupset     7183 non-null object
crankset_material     2493 non-null object
crankset_groupset     5008 non-null object
seatpost_material     4264 non-null object
chain_groupset        4712 non-null object
shifter_groupset      6913 non

#### Numerical
##### Price
* we have values less than 0, we should look into these and possibly drop before test split
* there seems to be few very expensive bike samples in the dataset; definitely want to ensure they're distributed in both train and test sets

##### Model Year
* range from 2013 to 2020
* looks like most bike sample are 2019 model year, which makes sense since data was collected in 2019-2020 season

In [16]:
munged_df.model_year.mode()

0    2020.0
dtype: float64

In [17]:
munged_df.describe()

,price,msrp,model_year
count,9558.000000,9596.000000,3831.000000
mean,2921.646361,3073.243260,2019.303576
std,2594.415468,2669.221096,0.858066
min,-1.000000,-1.000000,2013.000000
25%,749.990000,780.000000,2019.000000
50%,2200.000000,2429.000000,2019.000000
75%,4299.000000,4500.000000,2020.000000
max,16000.000000,16000.000000,2020.000000


### Data Cleaning
From quick glance of the dataset, it looks like it needs a little more data cleaning before we can proceed with test set hold out. Tasks:
1. Remove samples that missing values for all fields except "brand", "price", "brake_type" which have no missing values.
2. We have prices less than zero, we should drop samples with bike prices less than a certain threshold
3. Look at **bike_type** field and determine if we need to drop any specific values. We care primarily about adult bikes.

In [21]:
# Identify samples with empty rows
empty_rows_df = munged_df[(munged_df.frame_material.isnull()) & (munged_df.model_year.isnull())
         & (munged_df.fork_material.isnull()) & (munged_df.handlebar_material.isnull())
         & (munged_df.fd_groupset.isnull()) & (munged_df.rd_groupset.isnull())
         & (munged_df.cassette_groupset.isnull()) & (munged_df.crankset_material.isnull())
         & (munged_df.crankset_groupset.isnull()) & (munged_df.seatpost_material.isnull())
         & (munged_df.chain_groupset.isnull()) & (munged_df.shifter_groupset.isnull())]
empty_rows_df.head()

,site,bike_type,product_id,href,description,brand,price,msrp,frame_material,model_year,...,fork_material,handlebar_material,fd_groupset,rd_groupset,cassette_groupset,crankset_material,crankset_groupset,seatpost_material,chain_groupset,shifter_groupset
354,jenson,kid,BI223K01,/Strider-12-Sport-Kids-Balance-Bike,Strider 12 Sport Kids Balance Bike,Strider,109.99,119.99,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,jenson,kid,BI001267,/Strider-14x-Sport-Kids-Balance-Bike,Strider 14x Sport Kids Balance Bike,Strider,189.99,209.99,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,nashbar,gravel,1147420,https://www.nashbar.com/bombtrack-hook-2-cyclo...,Bombtrack Hook 2 Cyclocross/Gravel Bike (Black...,Bombtrack,2309.99,1999.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,nashbar,gravel,1151838,https://www.nashbar.com/ridley-xride-disc-riva...,Ridley X-Ride Disc Rival 1 Cyclocross Bike (Grey),Ridley,1669.00,1399.99,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
382,nashbar,cruiser,1147425,https://www.nashbar.com/bombtrack-outlaw-urban...,Bombtrack Outlaw Urban Bike (Black) (650B),Bombtrack,1498.99,1189.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# confirm they do indeed have empty values for other columns
empty_rows_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 354 to 9593
Data columns (total 21 columns):
site                  261 non-null object
bike_type             261 non-null object
product_id            261 non-null object
href                  261 non-null object
description           261 non-null object
brand                 261 non-null object
price                 261 non-null float64
msrp                  261 non-null float64
frame_material        0 non-null object
model_year            0 non-null float64
brake_type            0 non-null object
fork_material         0 non-null object
handlebar_material    0 non-null object
fd_groupset           0 non-null object
rd_groupset           0 non-null object
cassette_groupset     0 non-null object
crankset_material     0 non-null object
crankset_groupset     0 non-null object
seatpost_material     0 non-null object
chain_groupset        0 non-null object
shifter_groupset      0 non-null object
dtypes: float64(3), object(18)
me

In [8]:
# Identify bikes with prices less than 100
low_price_df = munged_df[munged_df.price < 100]
low_price_df.head()

,bike_type,brand,price,frame_material,model_year,brake_type,fork_material,handlebar_material,fd_groupset,rd_groupset,cassette_groupset,crankset_material,crankset_groupset,seatpost_material,chain_groupset,shifter_groupset
95,road,Bianchi,-1.0,carbon,2018.0,rim,carbon,NaN,shimano ultegra di2,shimano ultegra di2,shimano ultegra,NaN,shimano ultegra,NaN,NaN,shimano ultegra di2
666,mountain,Niner,-1.0,carbon,2017.0,hydraulic,NaN,NaN,NaN,shimano deore xt,shimano deore xt,NaN,shimano deore xt,NaN,shimano deore xt,shimano deore xt
667,mountain,Pivot,-1.0,carbon,2018.0,hydraulic,NaN,carbon,NaN,shimano xtr,shimano deore xt,NaN,race face,NaN,NaN,shimano deore xt
668,mountain,Pivot,-1.0,carbon,2017.0,hydraulic,NaN,carbon,NaN,sram xO1 eagle,sram xO1 eagle,NaN,sram xO1 eagle,carbon,NaN,sram xO1 eagle
715,mountain,Borealis,-1.0,carbon,2018.0,hydraulic,NaN,alloy,NaN,sram xO1 eagle,sram gx eagle,NaN,sram gx,alloy,sram x1,sram gx


In [9]:
# Identify bike_type values that should be dropped
munged_df.bike_type.value_counts()

mountain       781
road           441
childrens      242
urban          236
bmx            132
ebike          129
cyclocross      71
comfort         46
hybrid          40
touring         30
commuter        23
folding         12
triathlon        4
track            3
cargo            3
singlespeed      1
Name: bike_type, dtype: int64

In [10]:
# Drop children's bike and focus on adult bikes only
children_df = munged_df[munged_df.bike_type == 'childrens']
children_df.head()

,bike_type,brand,price,frame_material,model_year,brake_type,fork_material,handlebar_material,fd_groupset,rd_groupset,cassette_groupset,crankset_material,crankset_groupset,seatpost_material,chain_groupset,shifter_groupset
145,childrens,Fuji,159.99,alloy,2019.0,coaster,steel,steel,NaN,NaN,NaN,steel,NaN,alloy,NaN,NaN
198,childrens,Fuji,344.99,alloy,2019.0,linear_pull,NaN,steel,shimano tourney,shimano acera,shimano altus,NaN,NaN,steel,NaN,shimano tourney
207,childrens,Fuji,424.99,alloy,2019.0,mechanical,NaN,alloy,NaN,shimano acera,shimano tourney,NaN,NaN,alloy,NaN,shimano altus
213,childrens,Fuji,239.99,alloy,NaN,linear_pull,NaN,steel,NaN,shimano tourney,single speed,NaN,NaN,steel,NaN,shimano tourney
215,childrens,SE,169.00,NaN,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Create dataframe without dropped rows
drop_list = empty_rows_df.index.tolist() + low_price_df.index.tolist() + children_df.index.tolist()
drop_list = set(drop_list)  # remove dupes
drop_count = len(drop_list)
print(f'There are {drop_count} samples to be dropped.')
print(f'There should be {len(munged_df) - drop_count} samles left afterwards.')
data_df = munged_df.drop(drop_list)
print(f'There are now {len(data_df)} samples remaining.')

There are 274 samples to be dropped.
There should be 1920 samles left afterwards.
There are now 1920 samples remaining.


In [12]:
data_df.bike_type.value_counts()

mountain       769
road           438
urban          234
ebike          128
bmx            120
cyclocross      71
comfort         46
hybrid          39
touring         30
commuter        23
folding         12
triathlon        4
track            3
cargo            2
singlespeed      1
Name: bike_type, dtype: int64

In [13]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1920 entries, 0 to 2193
Data columns (total 16 columns):
bike_type             1920 non-null object
brand                 1920 non-null object
price                 1920 non-null float64
frame_material        1765 non-null object
model_year            1441 non-null float64
brake_type            1920 non-null object
fork_material         966 non-null object
handlebar_material    1032 non-null object
fd_groupset           621 non-null object
rd_groupset           1288 non-null object
cassette_groupset     1430 non-null object
crankset_material     563 non-null object
crankset_groupset     980 non-null object
seatpost_material     928 non-null object
chain_groupset        886 non-null object
shifter_groupset      1062 non-null object
dtypes: float64(2), object(14)
memory usage: 255.0+ KB


In [14]:
data_df.describe()

,price,model_year
count,1920.000000,1441.000000
mean,2520.567760,2018.252602
std,2163.364683,0.845763
min,159.990000,2013.000000
25%,724.990000,2018.000000
50%,1894.500000,2018.000000
75%,3599.990000,2019.000000
max,12519.990000,2020.000000


## Train Test Split
Implement "stratified" split for this regression data. By doing so, the hope is to get a more reliable (lower bias and variance) estimate of model performance. Using regular random split on the dataset is likely to naturally introduce bias into the data or unintentionally exclude data points, and thus affect model performance. Using stratification during model selection produces better results because the validation set(s) more accurately represent the task we need to solve. If one class is hard to predict, we won’t have more of them (artificially hindering the performance metric) or less (assisting performance) if we use stratification.

#### Sorted Stratification
Since this a regression task and not classification, we need a method to define the "classification" sets of the data so we can ensure similar distribution of the target variable in both the train and test sets. The following approach will be used to achieve this:
1. Sort the samples on the target variable
2. Categorize samples into *k* partitions (or "classes") depending on test size needed
3. Utilize sklearn's "StratifiedShuffleSplit" to generate tran vs test split

Alternative approach is to categorize samples by **bike_type** to ensure similar distribution of bike types between train and test sets. Categorizing by target variable is most likely the best since that is what we're predicting for. The latter might introduce other biases that we're unaware.

In [40]:
# Set random state for reproducibility purposes
RAND_STATE = 42

In [41]:
# We can simplify steps 1 and 2 by help of pandas.qcut method
price_cats = pd.qcut(x=data_df.price, q=10, labels=range(10))
price_cats.head()

0    9
1    9
2    9
3    9
4    9
Name: price, dtype: category
Categories (10, int64): [0 < 1 < 2 < 3 ... 6 < 7 < 8 < 9]

In [42]:
# Generate shuffled train and test indices
train_idx, test_idx = next(
    StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=RAND_STATE).split(
        X=price_cats, y=price_cats
    )
)
print(f'Num train samples: {len(train_idx)}\nNum test samples: {len(test_idx)}')

Num train samples: 1536
Num test samples: 384


In [51]:
# Get train and test datasets
train_df = data_df.iloc[train_idx].copy()
test_df = data_df.iloc[test_idx].copy()  # Hold-out set

# Verify dimensions
print(f'train shape: {train_df.shape}\ntest shape: {test_df.shape}')

train shape: (1536, 16)
test shape: (384, 16)


## Train Data Exploration and Feature Engineering
The data has many categorical fields. The first step is see how we can transform them into useful numerical fields. This will involving some one-hot encoding and identifying rankable values.

**price** and **model_year** are already numerical. Price is the target variable, so we won't directly touch this since we don't want to include it as a feature else we will be indirectly including what we want to predict as feature.

Let's look at each of the remaining categorical fields to see how best to utilize or whether to just drop them. Model year has some missing valaues so we will need to decide how we want to handle those missing values.

In [48]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1536 entries, 1687 to 1902
Data columns (total 16 columns):
bike_type             1536 non-null object
brand                 1536 non-null object
price                 1536 non-null float64
frame_material        1418 non-null object
model_year            1146 non-null float64
brake_type            1536 non-null object
fork_material         756 non-null object
handlebar_material    836 non-null object
fd_groupset           477 non-null object
rd_groupset           1027 non-null object
cassette_groupset     1149 non-null object
crankset_material     460 non-null object
crankset_groupset     795 non-null object
seatpost_material     741 non-null object
chain_groupset        717 non-null object
shifter_groupset      857 non-null object
dtypes: float64(2), object(14)
memory usage: 204.0+ KB


#### Frame Material
There are 1418 non-null samples. Looking at value counts, we see majority of frame material are either carbon or aluminium/alloy material. This is typical of what you would expect, majority of bikes this day are primarily aluminium or some alloy and for top-end there's a high demand for carbon due to its lightness and stiffness. The average price per frame material confirms this. Carbon frame bikes are considerably more expensive than aluminium or alloy frame bikes. Titanium also seems to garner higher price but we only have two samples represented in train dataset so we should consider trying to get more data for such bikes.

Since carbon and titanium seem to have a strong relationship to price, the best approach for this categorical data is to implement one-hot encoding for carbon frame material. The same will be done for titanium in preparation for when more data is collected. After doing some further research, titanium seems to be an upcoming frame material for similar reasons as carbon but less fragile.

In [52]:
print(train_df.frame_material.value_counts())
grouped = train_df.groupby('frame_material')
grouped['price'].mean()

carbon      590
aluminum    454
alloy       222
steel        88
chromoly     62
titanium      2
Name: frame_material, dtype: int64


frame_material
alloy       1350.845631
aluminum    1639.982753
carbon      4240.846712
chromoly    1002.245161
steel        777.092727
titanium    3499.000000
Name: price, dtype: float64

In [54]:
train_df['carbon_frame'] = train_df.frame_material.apply(lambda x: 1 if x == 'carbon' else 0)
train_df.carbon_frame.value_counts()

0    946
1    590
Name: carbon_frame, dtype: int64

In [53]:
train_df['titanium_frame'] = train_df.frame_material.map(lambda x: 1 if x == 'titanium' else 0)
train_df.titanium_frame.value_counts()

0    1534
1       2
Name: titanium_frame, dtype: int64

#### Other Material Categorical Fields
Except for fork material, we see that carbon material continues to have a strong relationship to price for other material based fields. So we will implement similar one-hot encoding to extract numerical features for these fields.

In [55]:
print(train_df.handlebar_material.value_counts())
grouped = train_df.groupby('handlebar_material')
grouped['price'].mean()

alloy       565
carbon      139
aluminum     73
steel        41
chromoly     18
Name: handlebar_material, dtype: int64


handlebar_material
alloy       1951.561398
aluminum    1734.927260
carbon      6430.066835
chromoly     658.993333
steel        418.381463
Name: price, dtype: float64

In [58]:
train_df['carbon_handlebar'] = train_df.handlebar_material.apply(lambda x: 1 if x == 'carbon' else 0)
train_df.carbon_handlebar.value_counts()

0    1397
1     139
Name: carbon_handlebar, dtype: int64

In [56]:
print(train_df.crankset_material.value_counts())
grouped = train_df.groupby('crankset_material')
grouped['price'].mean()

alloy       245
carbon       79
steel        55
aluminum     48
chromoly     33
Name: crankset_material, dtype: int64


crankset_material
alloy       1603.336286
aluminum    2626.796667
carbon      6350.193418
chromoly     461.870000
steel       1159.007273
Name: price, dtype: float64

In [59]:
train_df['carbon_crankset'] = train_df.crankset_material.apply(lambda x: 1 if x == 'carbon' else 0)
train_df.carbon_crankset.value_counts()

0    1457
1      79
Name: carbon_crankset, dtype: int64

In [57]:
print(train_df.seatpost_material.value_counts())
grouped = train_df.groupby('seatpost_material')
grouped['price'].mean()

alloy       517
carbon      156
aluminum     48
steel        20
Name: seatpost_material, dtype: int64


seatpost_material
alloy       1299.749729
aluminum    1622.093125
carbon      4037.964103
steel        476.789500
Name: price, dtype: float64

In [60]:
train_df['carbon_seatpost'] = train_df.seatpost_material.apply(lambda x: 1 if x == 'carbon' else 0)
train_df.carbon_seatpost.value_counts()

0    1380
1     156
Name: carbon_seatpost, dtype: int64

Alloy, aluminium, and carbon fork have relatively same average price. Surprisingly, having steel and chromoly fork has strong negative relationship to price. For this, we will implement one-hot encoding for steel and chromoly fork material; or rather one-hot encoding for non-carbon, alloy, aluminium for material.

In [61]:
print(train_df.fork_material.value_counts())
grouped = train_df.groupby('fork_material')
grouped['price'].mean()

carbon      396
steel       164
alloy        73
chromoly     67
aluminum     56
Name: fork_material, dtype: int64


fork_material
alloy       2542.837123
aluminum    2259.267143
carbon      2783.785833
chromoly    1011.839104
steel        847.309939
Name: price, dtype: float64

In [65]:
train_df['nonstandard_fork'] = train_df.fork_material.apply(lambda x: 1 if x == 'chromoly' or x == 'steel' else 0)
train_df.nonstandard_fork.value_counts()

0    1305
1     231
Name: nonstandard_fork, dtype: int64

#### Brake Type

In [66]:
print(train_df.brake_type.value_counts())
grouped = train_df.groupby('brake_type')
grouped['price'].mean()

hydraulic      773
other          298
mechanical     125
disc           114
vbrake          59
rim             53
caliper         48
linear_pull     28
ubrake          20
coaster         18
Name: brake_type, dtype: int64


brake_type
caliper        1373.881458
coaster         384.374444
disc           2420.468070
hydraulic      3310.022885
linear_pull     996.412143
mechanical     1089.069840
other          2097.056946
rim            2780.842264
ubrake          310.039500
vbrake          499.992203
Name: price, dtype: float64

In [68]:
train_df[(train_df.brake_type == 'disc') | (train_df.brake_type == 'hydraulic') | (train_df.brake_type == 'mechanical')].mean()

price               2935.488913
model_year          2018.330544
titanium_frame         0.000000
carbon_frame           0.443676
carbon_handlebar       0.113636
carbon_crankset        0.070158
carbon_seatpost        0.103755
nonstandard_fork       0.072134
dtype: float64

In [69]:
train_df[train_df.brake_type == 'other'] ### DATA INTEGRITY ISSUES - NEED TO CLEAN UP DATA PIPELINE
# Seem to have framesets in samples and unnecessarily incomplete samples which have actual specs on website

,bike_type,brand,price,frame_material,model_year,brake_type,fork_material,handlebar_material,fd_groupset,rd_groupset,...,crankset_groupset,seatpost_material,chain_groupset,shifter_groupset,titanium_frame,carbon_frame,carbon_handlebar,carbon_crankset,carbon_seatpost,nonstandard_fork
2046,ebike,Specialized,3449.99,NaN,2019.0,other,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
1699,mountain,Santa Cruz,2799.99,carbon,2019.0,other,NaN,NaN,NaN,sram nx,...,sram nx,NaN,sram nx,sram nx,0,1,0,0,0,0
1825,mountain,Haro,329.99,NaN,2017.0,other,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
910,track,Fuji,336.99,chromoly,2018.0,other,chromoly,alloy,NaN,NaN,...,NaN,alloy,NaN,NaN,0,0,0,0,0,1
1535,road,Specialized,4124.99,carbon,2017.0,other,carbon,alloy,sram red etap,sram red etap,...,NaN,carbon,NaN,sram red etap,0,1,0,0,1,0
2061,bmx,Haro,1369.99,chromoly,2018.0,other,chromoly,chromoly,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,1
1616,road,Specialized,679.99,aluminum,2016.0,other,carbon,alloy,shimano sora,shimano sora,...,shimano sora,alloy,sram x9,shimano sora,0,0,0,0,0,0
676,cyclocross,Pivot,4499.00,carbon,NaN,other,carbon,NaN,shimano ultegra,shimano ultegra,...,praxis,NaN,NaN,shimano ultegra,0,1,0,0,0,0
1890,mountain,Specialized,2499.99,NaN,2019.0,other,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2193,bmx,Kink,699.99,chromoly,2019.0,other,chromoly,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,1


#### Model Year Field

In [ ]:
# Summary statistics
train_df.model_year.describe()

In [ ]:
# Mode
train_df.model_year.mode()

In [ ]:
# Counts by values
train_df.model_year.value_counts()

The mean is about 2018, the median is 2018, and the mode is 2019. Let's set missing values to 2018

In [ ]:
train_df[train_df.model_year.isnull()]